In [ ]:
# @title # RunFlux

# @markdown ## Train a FLUX1-dev LoRA on RunPod.

# @markdown * 📸 **Caption 10-15 images**, upload to Google Drive or Dropbox

# @markdown * ☁️ **Choose a GPU** on RunPod

# @markdown * 🤗 Choose a **Hugging Face target repo**

# @markdown * 🔑 **Insert API tokens** for Hugging Face and RunPod

# @markdown * 🔑 Accept the FLUX1-dev licence: https://huggingface.co/black-forest-labs/FLUX.1-dev

# @markdown * 🪄 **Click RUN to start your Pod**. LoRA adapters and sample images will be uploaded to your HF repo while training. The pod will shut down when training is complete.

# @markdown 💪💪💪 *This notebook trains with the `ai-toolkit` created by Ostris. Make sure to give him a star and follow*:
# @markdown * https://github.com/ostris/ai-toolkit
# @markdown * https://x.com/ostrisai

# @markdown ---

# @markdown ## 📸 Images and captions
IMAGE_ARCHIVE = "" # @param {type:"string"}

# @markdown * Ideally use a **Google Drive** or **Dropbox** share link, it will be converted to a direct download link. If it's not a Google/Dropbox URL, make sure it is a **direct download link**.

# @markdown * The archive has to be a `.zip` containing image and text files:
# @markdown * For example: `DSCN1744.JPG` and `DSCN1744.txt`
# @markdown * The `.txt` file contains the caption of each image. For example: "`p3r5on with red hair, playing chess at the park, bomb going off in the background`"
# @markdown * 10-15 images are enough

TRIGGER_WORD = "p3r5on" # @param {type:"string"}


# @markdown ---

# @markdown ## 💪 Training Parameters
# @markdown Leave the defaults for a first try.

LORA_RANK = 16 # @param {type:"integer"}
LORA_ALPHA = 16 # @param {type:"integer"}
BATCH_SIZE = 1 # @param {type:"integer"}
SAMPLE_STEPS = 500 # @param {type:"integer"}
SAVE_STEPS = 500 # @param {type:"integer"}
TRAIN_STEPS = 3000 # @param {type:"integer"}
LEARNING_RATE = 0.0001 # @param {type:"number"}
QUANTIZE_MODEL = True # @param {type:"boolean"}
SEED = 42 # @param {type:"integer"}


# @markdown ---

# @markdown ## 🤗 Output
# @markdown This HF repo will be created and the LoRA adapters uploaded once training is finished.

HF_REPO = "user/repo" # @param {type:"string"}

# @markdown ---

# @markdown ## ☁️ Cloud GPU

# @markdown Choose a GPU with at least 24 GB VRAM, check https://www.runpod.io/pricing

GPU = "NVIDIA GeForce RTX 4090" # @param ["NVIDIA A100 80GB PCIe", "NVIDIA A100-SXM4-80GB", "NVIDIA A30", "NVIDIA A40", "NVIDIA GeForce RTX 3090", "NVIDIA GeForce RTX 3090 Ti", "NVIDIA GeForce RTX 4090", "NVIDIA H100 80GB HBM3", "NVIDIA H100 NVL", "NVIDIA H100 PCIe", "NVIDIA L4", "NVIDIA L40", "NVIDIA L40S", "NVIDIA RTX 5000 Ada Generation", "NVIDIA RTX 6000 Ada Generation", "NVIDIA RTX A5000", "NVIDIA RTX A6000", "Tesla V100-SXM2-32GB"]
CLOUD_TYPE = "SECURE" # @param ["COMMUNITY", "SECURE"]
POD_NAME = "FLUX1-DEV Train LoRa" # @param {type:"string"}

# @markdown ---

# @markdown ## 🔑 Tokens
# @markdown Create Runopd and Hugging Face tokens and insert here.

# @markdown * https://www.runpod.io/console/user/settings (**Read & Write** token)

# @markdown * https://huggingface.co/settings/tokens  (**Write** token)

RUNPOD_TOKEN = "" # @param {type:"string"}
HF_TOKEN = "" # @param {type:"string"}

# @markdown ---

# @markdown ## 🚨 Before you start

# @markdown Sign into HuggingFace and accept the FLUX1-dev access: https://huggingface.co/black-forest-labs/FLUX.1-dev


!pip install -qqq runpod --progress-bar off
import runpod, re

runpod.api_key = RUNPOD_TOKEN
HUGGINGFACE_TOKEN = HF_TOKEN

def to_direct_link(url):
  def extract(url, pattern):
      match = re.search(pattern, url)
      return match.group(1) if match else None

  if "drive.google.com" in url:
    if "id=" in url:
      fileId = extract(url, r'id=([a-zA-Z0-9_-]+)')
    else:
      fileId = extract(url, r'/d/([a-zA-Z0-9_-]+)')
    return f'https://drive.google.com/uc?export=download&id={fileId}' if fileId else None
  elif "dropbox.com" in url:
    return url.replace("dl=0", "dl=1")
  else:
    return url

IMAGE_ARCHIVE_DIRECTLINK = to_direct_link(IMAGE_ARCHIVE)

assert IMAGE_ARCHIVE_DIRECTLINK is not None, "Can't parse URL"
assert len(RUNPOD_TOKEN) > 0
assert len(HF_TOKEN) > 0
assert len(HF_REPO) > 0

pod = runpod.create_pod(
    name=POD_NAME,
    image_name="runpod/pytorch:2.2.0-py3.10-cuda12.1.1-devel-ubuntu22.04",
    gpu_type_id=GPU,
    cloud_type=CLOUD_TYPE,
    gpu_count=1,
    container_disk_in_gb=50,
    volume_in_gb=10,
    docker_args=f'bash -c \'cd /workspace; git clone https://github.com/geronimi73/RunFlux; bash /workspace/RunFlux/runpod.sh\'',
    volume_mount_path="/workspace",
    env={
        "HF_REPO": HF_REPO,
        "HUGGINGFACE_TOKEN": HF_TOKEN,
        "IMAGE_ARCHIVE": IMAGE_ARCHIVE_DIRECTLINK,
        "TRIGGER_WORD": TRIGGER_WORD,
        "LORA_RANK": LORA_RANK,
        "LORA_ALPHA": LORA_ALPHA,
        "BATCH_SIZE": BATCH_SIZE,
        "SAMPLE_STEPS": SAMPLE_STEPS,
        "SAVE_STEPS": SAVE_STEPS,
        "STEPS": TRAIN_STEPS,
        "LEARNING_RATE": LEARNING_RATE,
        "QUANTIZE_MODEL": QUANTIZE_MODEL,
        "SEED": SEED,
    }
)


print("Pod started: https://www.runpod.io/console/pods")


Pod started: https://www.runpod.io/console/pods
